<a href="https://colab.research.google.com/github/MacUpr/im_ia/blob/main/Imers%C3%A3o_Agentes_de_IA_Alura_%2B_Google_Gemini_ipynb_Aula_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.2 MB/s eta 0:00:00


Importação da API Key

In [2]:
book on from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

SecretNotFoundError: Secret GEMINI_API_KEY does not exist.

Conexão com o Gemini

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("Quem é você? Seja criativo.")
print(resp_test.content)

In [ ]:
TRIAGEM_PROMPT = (
    "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Seu objetivo é classificar as mensagens dos usuários em uma das seguintes categorias:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Considere as seguintes regras para a classificação:\n"
    '- **AUTO_RESOLVER**: Use esta opção quando a pergunta for clara e puder ser respondida diretamente com base nas políticas internas. Exemplos incluem perguntas sobre reembolso de despesas, procedimentos de trabalho remoto, ou outras regras documentadas.\n'
    '- **PEDIR_INFO**: Use esta opção quando a mensagem for vaga, ambígua, ou faltarem informações cruciais para entender a solicitação. Inclua na lista `campos_faltantes` quais informações adicionais são necessárias.\n'
    '- **ABRIR_CHAMADO**: Use esta opção para solicitações que exigem ação ou aprovação de um departamento específico (como RH, TI, Financeiro) ou quando o usuário explicitamente pede para abrir um chamado. Exemplos incluem pedidos de exceção a políticas, liberação de acesso, ou solicitações que requerem intervenção humana.\n'
    "Analise cuidadosamente a mensagem do usuário e forneça a classificação no formato JSON especificado."
)

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
    decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"] = Field(description="A decisão da triagem baseada na mensagem do usuário.")
    urgencia: Literal["BAIXA", "MEDIA", "ALTA"] = Field(description="O nível de urgência da solicitação do usuário.")
    campos_faltantes: List[str] = Field(default_factory=list, description="Lista de campos ou informações que faltam na mensagem do usuário para processar a solicitação.")

In [ ]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    try:
        saida: TriagemOut = triagem_chain.invoke([
            SystemMessage(content=TRIAGEM_PROMPT),
            HumanMessage(content=mensagem)
        ])
        return saida.model_dump()
    except Exception as e:
        print(f"Error parsing structured output: {e}")
        return {"error": "Could not parse structured output."}

In [ ]:
testes = [
    "Posso reembolsar a internet?",
    "Quero mais 5 dias de trabalho remoto. Como faço?",
    "Posso reembolsar cursos ou treinamentos da Alura?",
    "Quantas capivaras tem no Rio Pinheiros?",
    "Preciso de um novo mouse, o meu parou de funcionar.", # Exemplo para ABRIR_CHAMADO (TI)
    "Qual o procedimento para solicitar férias?", # Exemplo para AUTO_RESOLVER (RH/Política)
    "Minha solicitação de reembolso ainda não foi aprovada.", # Exemplo para ABRIR_CHAMADO (Financeiro/Follow-up)
    "Gostaria de saber sobre os benefícios de plano de saúde.", # Exemplo para AUTO_RESOLVER (RH/Política)
    "Preciso de ajuda com um problema no sistema X, não consigo acessar meus dados.", # Exemplo para ABRIR_CHAMADO (TI)
    "Como faço para atualizar minhas informações de contato?", # Exemplo para AUTO_RESOLVER (RH/Procedimento)
    "Tenho uma dúvida sobre a política de viagens, a empresa cobre alimentação?", # Exemplo para AUTO_RESOLVER (Política)
    "Não entendi a última comunicação sobre as novas diretrizes de segurança, podem explicar melhor?", # Exemplo para PEDIR_INFO
    "Preciso de acesso ao sistema Y para o projeto Z.", # Exemplo para ABRIR_CHAMADO (TI/Acesso)
    "Qual o prazo para reembolso de despesas de viagem?", # Exemplo para AUTO_RESOLVER (Política/Prazo)
    "Tive um problema com meu crachá, ele não está funcionando.", # Exemplo para ABRIR_CHAMADO (TI/Hardware)
    "Gostaria de saber se a empresa oferece algum tipo de auxílio para desenvolvimento profissional além dos treinamentos obrigatórios.", # Exemplo para AUTO_RESOLVER (RH/Política)
    "A internet na minha casa está muito lenta, isso afeta meu trabalho remoto. O que posso fazer?", # Exemplo para ABRIR_CHAMADO (TI/Suporte)
    "Preciso de uma declaração de vínculo empregatício.", # Exemplo para ABRIR_CHAMADO (RH/Documento)
    "Qual o horário de funcionamento do escritório?", # Exemplo para AUTO_RESOLVER (Geral)
    "Não consigo instalar o software Z no meu notebook da empresa.", # Exemplo para ABRIR_CHAMADO (TI/Software)
]

In [ ]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}")
    resposta = triagem(msg_teste)
    if "error" in resposta:
        print(f" -> Resposta: {resposta['error']}\n")
    else:
        print(f" -> Decisão: {resposta['decisao']}")
        print(f" -> Urgência: {resposta['urgencia']}")
        if resposta['campos_faltantes']:
            print(f" -> Campos Faltantes: {', '.join(resposta['campos_faltantes'])}")
        print()